In [1]:
!ls videos/IMG_5303  

IMG_5303.zip             IMG_5303_en.srt          IMG_5303_zh.md
IMG_5303_cover.jpg       IMG_5303_highlighted.mp4 IMG_5303_zh.srt
IMG_5303_en.md           IMG_5303_metadata.json


In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from pathlib import Path
import logging
import platform
import json
import json5

# Setup WebDriver and constants
video_path = "videos/IMG_5303/IMG_5303_highlighted.mp4"
thumbnail_path = "videos/IMG_5303/IMG_5303_cover.jpg"

# Load metadata from the JSON file
metadata_file_path = "videos/IMG_5303/IMG_5303_metadata.json"
try:
    with open(metadata_file_path, 'r') as file:
        metadata_dict = json.load(file)
    print('Metadata loaded successfully.')
except Exception as e:
    print(f"An error occurred while loading the metadata: {e}")
metadata_dict

YOUTUBE_UPLOAD_URL = 'https://www.youtube.com/upload'
USER_WAITING_TIME = 5  # Adjust as needed
chrome_debugging_port = "9222"

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Check if the operating system is Mac
is_mac = not any(os_name in platform.platform() for os_name in ["Windows", "Linux"])

# Set up Chrome options to connect to the remote debugging port
chrome_options = ChromeOptions()
chrome_options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debugging_port}")
browser = Chrome(options=chrome_options)

logger.debug("Connected to Chrome on port: {}".format(chrome_debugging_port))

# Rest of your script (uploading video to YouTube)...

browser.get(YOUTUBE_UPLOAD_URL)
time.sleep(USER_WAITING_TIME)

# Dismiss alert if present
try:
    alert = browser.switch_to.alert
    alert.accept()  # Use alert.accept() to accept the alert.
    print("Alert was present and dismissed.")
except:
    print("No alert present.")

# Upload the video file
absolute_video_path = str(Path.cwd() / video_path)
browser.find_element(By.XPATH, "//input[@type='file']").send_keys(absolute_video_path)
print('Attached video {}'.format(video_path))

# Define the text to look for in the span element
expected_text = "Checks complete. No issues found."

# Wait for the presence of the span element containing the expected text
try:
    wait = WebDriverWait(browser, 30)  # Adjust timeout as needed
    span_xpath = f"//span[contains(@class, 'progress-label') and contains(@class, 'style-scope') and contains(@class, 'ytcp-video-upload-progress') and contains(text(), '{expected_text}')]"
    wait.until(EC.presence_of_element_located((By.XPATH, span_xpath)))
    print('The expected text is present in the span element.')
except Exception as e:
    print(f"An error occurred while waiting for the text: {e}")


# Prepare the video title with tags
video_title_with_tags = metadata_dict["title"] + " " + " ".join("#" + tag for tag in metadata_dict["tags"])
print(f'Video title with tags: {video_title_with_tags}')

# Wait for the video title input field to be clickable
title_input_xpath = "//div[@id='textbox'][@contenteditable='true']"
wait = WebDriverWait(browser, 20)
title_input = wait.until(EC.element_to_be_clickable((By.XPATH, title_input_xpath)))

# Clear the input field, click on it, and then set the video title with tags
title_input.clear()
title_input.click()
title_input.send_keys(video_title_with_tags)
print(f'The video title was set to "{video_title_with_tags}"')

# Enter video description
description_input_xpath = "//div[@id='textbox'][@contenteditable='true' and @aria-label='Tell viewers about your video (type @ to mention a channel)']"
description_input = wait.until(EC.element_to_be_clickable((By.XPATH, description_input_xpath)))
description_input.clear()
description_input.click()
description_input.send_keys(metadata_dict["long_description"])
print(f'The video description was set to "{metadata_dict["long_description"]}"')

# Set video thumbnail
try:
    infobox_message = browser.find_element(By.CSS_SELECTOR, ".infobox-container .style-scope.ytcp-thumbnails-compact-editor").text
    if "you can’t change the thumbnail on your Short" in infobox_message:
        print('Skipping thumbnail upload because it is not allowed for this video type.')
    else:
        absolute_thumbnail_path = str(Path.cwd() / thumbnail_path)
        browser.find_element(By.XPATH, "//input[@id='file-loader']").send_keys(absolute_thumbnail_path)
        print('Attached thumbnail {}'.format(thumbnail_path))
except NoSuchElementException:
    absolute_thumbnail_path = str(Path.cwd() / thumbnail_path)
    browser.find_element(By.XPATH, "//input[@id='file-loader']").send_keys(absolute_thumbnail_path)
    print('Attached thumbnail {}'.format(thumbnail_path))



# Add video to playlist (if specified)
playlist_name = "SimpleLife"
try:
    dropdown_trigger = browser.find_element(By.CSS_SELECTOR, "ytcp-text-dropdown-trigger.dropdown")
    dropdown_trigger.click()
    print('Clicked on playlist dropdown')

    wait = WebDriverWait(browser, 10)
    option_xpath = f"//span[contains(@class, 'style-scope') and text()='{playlist_name}']"
    playlist_option = wait.until(EC.visibility_of_element_located((By.XPATH, option_xpath)))
    playlist_option.click()
    print(f'Selected playlist: {playlist_name}')
    
except Exception as e:
    print(f"An error occurred during playlist selection: {e}")

# Now, click the 'Done' button after selecting the playlist
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    done_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "ytcp-button.done-button")))
    done_button.click()
    print('Clicked on the Done button.')
except Exception as e:
    print(f"An error occurred when trying to click on the Done button: {e}")


# Click the 'Not Made for Kids' radio button
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    not_made_for_kids_button = wait.until(EC.element_to_be_clickable((By.NAME, "VIDEO_MADE_FOR_KIDS_NOT_MFK")))
    not_made_for_kids_button.click()
    print('Clicked on the Not Made for Kids button.')
except Exception as e:
    print(f"An error occurred when trying to click on the Not Made for Kids button: {e}")

# Click the 'Show more' button
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    show_more_button = wait.until(EC.element_to_be_clickable((By.ID, "toggle-button")))
    show_more_button.click()
    print('Clicked on the Show more button.')
except Exception as e:
    print(f"An error occurred when trying to click on the Show more button: {e}")



# Define your tags
tags = metadata_dict["tags"]  # Replace with your actual tags
tags_string = ', '.join(tags) + ','  # Join the tags with a comma and space, add a trailing comma

# Find the tag input field and enter the tags
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    tags_input_xpath = "//input[@id='text-input' and @class='text-input style-scope ytcp-chip-bar' and @aria-label='Tags']"
    tags_input = wait.until(EC.element_to_be_clickable((By.XPATH, tags_input_xpath)))
    tags_input.click()  # Focus on the input
    tags_input.send_keys(tags_string)  # Type the tags
    print(f'Tags entered: {tags_string}')
except Exception as e:
    print(f"An error occurred when trying to enter tags: {e}")
    

# Click the 'Next' button
try:
    next_button = browser.find_element(By.ID, "next-button")
    if next_button.get_attribute("aria-disabled") == "false":
        next_button.click()
        print('Clicked on the Next button.')
    else:
        print('Next button is not clickable.')
except Exception as e:
    print(f"An error occurred when trying to click on the Next button: {e}")

# Click the 'Next' button
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    next_button = wait.until(EC.element_to_be_clickable((By.ID, "next-button")))
    next_button.click()
    print('Clicked on the Next button.')
except Exception as e:
    print(f"An error occurred when trying to click on the Next button: {e}")

# Click the 'Next' button
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    next_button = wait.until(EC.element_to_be_clickable((By.ID, "next-button")))
    next_button.click()
    print('Clicked on the Next button.')
except Exception as e:
    print(f"An error occurred when trying to click on the Next button: {e}")

# # Set video visibility to Public
# browser.find_element(By.NAME, 'PUBLIC').click()



# Click on the 'Next' button until you reach the 'Visibility' page
for _ in range(3):
    browser.find_element(By.ID, 'next-button').click()
    time.sleep(USER_WAITING_TIME)

Metadata loaded successfully.
No alert present.
Attached video videos/IMG_5303/IMG_5303_highlighted.mp4
An error occurred while waiting for the text: Message: 

Video title with tags: 午餐困境：食不下嚥？ #午餐挑戰 #味覺考驗 #食物選擇 #生活困境 #味覺極限 #幽默日常 #飲食探險 #美食困惑 #日常生活 #食物評價
The video title was set to "午餐困境：食不下嚥？ #午餐挑戰 #味覺考驗 #食物選擇 #生活困境 #味覺極限 #幽默日常 #飲食探險 #美食困惑 #日常生活 #食物評價"
The video description was set to "每個人的午餐時光都應該是一段愉快的時光，然而有時候它卻會變成一場味蕾的考驗。本影片展示了我們的主角在飢餓的驅動下，面對眼前令人卻步的食物。從食物的外觀到其味道，都向觀眾傳達出一種“看起來不好吃，聞起來也不好聞”的感覺。面對這樣的午餐，主角是否能夠咽下這頓飯？這段既幽默又引人思考的視頻將帶給觀眾深刻的觀看體驗。"
Skipping thumbnail upload because it is not allowed for this video type.
Clicked on playlist dropdown
Selected playlist: SimpleLife
Clicked on the Done button.
Clicked on the Not Made for Kids button.
Clicked on the Next button.
Clicked on the Next button.
Clicked on the Next button.
An error occurred when trying to click on the Show more button: Message: 

An error occurred when trying to enter tags: Message: 



ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=120.0.6099.234)


In [5]:
# Set the video's visibility and publish
browser.find_element(By.NAME, 'PUBLIC').click()

In [ ]:
# Click the 'Publish' button
try:
    wait = WebDriverWait(browser, 10)  # Adjust timeout as needed
    publish_button = wait.until(EC.element_to_be_clickable((By.ID, "done-button")))
    publish_button.click()
    print('Clicked on the Publish button.')
    # print("Video published successfully.")
except Exception as e:
    print(f"An error occurred when trying to click on the Publish button: {e}")

# Retrieve video ID (This assumes you are on the video's page after uploading)
video_id = browser.current_url.split('/')[-1]
print(f'Video ID: {video_id}')

# Close the browser
browser.quit()